In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
songs_df = pd.read_csv('Data/songs_w_genres_df.csv', index_col='id')
songs_df.head()

,name,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,,,,,,,,,,,,,,,,,,
0cS0A1fUEUd1EW3FcF8AEI,Keep A Song In Your Soul,Mamie Smith,1920,0,12,168333,0.991000,0.598,0.224,0.000522,0.3790,-12.628,0,F,0.0936,149.976,0.6340,"['harlem renaissance', 'traditional blues']"
0hbkKFIJm7Z05H8Zl9w30f,I Put A Spell On You,Screamin' Jay Hawkins,1920,0,7,150200,0.643000,0.852,0.517,0.026400,0.0809,-7.261,0,F,0.0534,86.889,0.9500,"['blues', 'traditional blues']"
11m7laMUgmOKqI3oYzuhne,Golfing Papa,Mamie Smith,1920,0,4,163827,0.993000,0.647,0.186,0.000018,0.5190,-12.098,1,C,0.1740,97.600,0.6890,"['harlem renaissance', 'traditional blues']"
19Lc5SfJJ5O1oaxY0fpwfh,True House Music - Xavier Santos & Carlos Gomi...,Oscar Velazquez,1920,0,17,422087,0.000173,0.730,0.798,0.801000,0.1280,-7.311,1,D,0.0425,127.997,0.0422,['circuit']
2hJjbsLCytGsnAHfdsLejp,Xuniverxe,Mixe,1920,1,2,165224,0.295000,0.704,0.707,0.000246,0.4020,-6.036,0,A#,0.0768,122.076,0.2990,['glitchcore']


In [3]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174389 entries, 0cS0A1fUEUd1EW3FcF8AEI to 57tgYkWQTNHVFEt6xDKKZj
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              174389 non-null  object 
 1   artists           174389 non-null  object 
 2   year              174389 non-null  int64  
 3   explicit          174389 non-null  int64  
 4   popularity        174389 non-null  int64  
 5   duration_ms       174389 non-null  int64  
 6   acousticness      174389 non-null  float64
 7   danceability      174389 non-null  float64
 8   energy            174389 non-null  float64
 9   instrumentalness  174389 non-null  float64
 10  liveness          174389 non-null  float64
 11  loudness          174389 non-null  float64
 12  mode              174389 non-null  int64  
 13  key               174389 non-null  object 
 14  speechiness       174389 non-null  float64
 15  tempo             174389 non-null  f

In [4]:
songs_df.index

Index(['0cS0A1fUEUd1EW3FcF8AEI', '0hbkKFIJm7Z05H8Zl9w30f',
       '11m7laMUgmOKqI3oYzuhne', '19Lc5SfJJ5O1oaxY0fpwfh',
       '2hJjbsLCytGsnAHfdsLejp', '3HnrHGLE9u2MjHtdobfWl9',
       '5DlCyqLyX2AOVDTjjkDZ8x', '02FzJbHtqElixxCmrpSCUa',
       '02i59gYdjlhBmbbWhf8YuK', '06NUxS2XL3efRh0bloxkHm',
       ...
       '45XnLMuqf3vRfskEAMUeCH', '1Cbf6PLWsL4s51eFepXx6L',
       '4pPFI9jsguIh3wC7Otoyy8', '3ASGdyWXeXsXtOIWtm0tv4',
       '52YtxLVUyvtiGPxwwxayHZ', '46LhBf6TvYjZU2SMvGZAbn',
       '7tue2Wemjd0FZzRtDrQFZd', '48Qj61hOdYmUCFJbpQ29Ob',
       '1gcyHQpBQ1lfXGdhZmWrHP', '57tgYkWQTNHVFEt6xDKKZj'],
      dtype='object', name='id', length=174389)

In [5]:
songs_df.shape

(174389, 18)

In [6]:
songs_1970s_up = songs_df[songs_df['year'] > 2000]
songs_1970s_up.year.unique()

array([2008, 2018, 2020, 2006, 2012, 2011, 2007, 2019, 2013, 2001, 2017,
       2016, 2010, 2002, 2004, 2005, 2015, 2003, 2009, 2021, 2014],
      dtype=int64)

# Cosine Similarity

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
songs_1970s_up.reset_index(inplace=True)
songs_1970s_up.set_index('name', inplace=True)


In [9]:
count = CountVectorizer()
count_matrix = count.fit_transform(songs_1970s_up['artists'])

#songs_1970s_up_acoust = songs_1970s_up.acousticness
#songs_acoust = songs_1970s_up_acoust.to_numpy()

#songs_acoust = songs_acoust.reshape(-1, 1)

#cosine_sim = cosine_similarity(songs_acoust, songs_acoust)

cosine_sim = cosine_similarity(count_matrix, count_matrix)

cosine_sim


array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.]])

In [10]:
indices = pd.Series(songs_1970s_up.index)
indices

0            You'll Never Walk Alone - Mono; 2002 Remaster
1             Ferry Cross the Mersey - Mono; 2002 Remaster
2         Don't Let the Sun Catch You Crying (Main) - Mono
3        The September Of My Years - Live At The Sands ...
4        It Was A Very Good Year - Live At The Sands Ho...
                               ...                        
41200                                              The One
41201                                        A Little More
41202                                             Together
41203                                   champagne problems
41204                                       Improvisations
Name: name, Length: 41205, dtype: object

In [26]:
def recommendations(id, cosine_sim=cosine_sim):
    
    idx = indices[indices == id].index[0]
    
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]
    
    songs_index = [i[0] for i in similarity_scores]
    
    return songs_1970s_up.iloc[songs_index]
    

In [27]:
recommendations('One Time', cosine_sim)

,id,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
name,,,,,,,,,,,,,,,,,,
One Less Lonely Girl,69ghzc538EQSVon2Gm3wrr,Justin Bieber,2009,0,68,229107,0.1590,0.580,0.750,0.0,0.2740,-4.421,1,E,0.1940,86.938,0.728,"['canadian pop', 'pop', 'post-teen pop']"
Mistletoe,7xapw9Oy21WpfEcib2ErSA,Justin Bieber,2011,0,84,182947,0.4980,0.657,0.560,0.0,0.0794,-9.049,0,F#,0.0467,161.993,0.854,"['canadian pop', 'pop', 'post-teen pop']"
Christmas Love,3RH9idbxUAMcUldet2ormp,Justin Bieber,2011,0,69,206787,0.5180,0.621,0.512,0.0,0.3280,-8.577,1,A#,0.0300,99.887,0.372,"['canadian pop', 'pop', 'post-teen pop']"
Love Yourself,50kpGaPAhYJ3sGmk6vplg0,Justin Bieber,2015,0,81,233720,0.8350,0.609,0.378,0.0,0.2800,-9.828,1,E,0.4380,100.418,0.515,"['canadian pop', 'pop', 'post-teen pop']"
Sorry,09CtPGIpYB4BrO8qb1RGsF,Justin Bieber,2015,0,81,200787,0.0797,0.654,0.760,0.0,0.2990,-3.669,0,C,0.0450,99.945,0.410,"['canadian pop', 'pop', 'post-teen pop']"


# Euclidean Distance

In [14]:
from sklearn.metrics.pairwise import euclidean_distances


In [21]:
D = euclidean_distances(count_matrix)

In [40]:
def recommend_euclidean_distance(name):
    
    idx = indices[indices == name].index[0]
    distance = list(enumerate(D[idx]))
    distance = sorted(distance, key=lambda x:x[1])
    distance = distance[1:6]
    
    songs_index = [i[0] for i in distance]
    
    return songs_df.iloc[songs_index]

In [43]:
recommend_euclidean_distance('One Time')

,name,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,,,,,,,,,,,,,,,,,,
1EZxS9QGorIenV1rthL2Zn,Mucamita - Remasterizado,"Francisco Canaro', 'Charlo",1929,0,0,163000,0.994,0.726,0.203,0.180000,0.520,-8.754,1,F,0.0862,120.554,0.862,NaN
0ChV818aHkgUpq8AeqxtLu,Oloi oi rempetes tou ntounia,Markos Vamvakaris,1930,0,0,194013,0.920,0.606,0.447,0.020400,0.091,-9.443,1,C#,0.0370,147.714,0.825,"['bouzouki', 'rebetiko']"
0kHmHERgdPewP1Gqbu0Nsi,Часть 30.9 & Часть 31.1 - Обратный путь,Эрих Мария Ремарк,1931,1,0,140697,0.791,0.710,0.177,0.000000,0.182,-15.940,0,G#,0.9550,109.011,0.562,[]
0HFqUkRuP4G49GpwqWICVv,Doin' The Jive,Glenn Miller,1935,0,35,180827,0.938,0.617,0.154,0.000058,0.172,-23.122,0,F,0.3410,184.826,0.903,"['adult standards', 'big band', 'easy listenin..."
1j3oXJcbQdarDIMWq91yKo,Gwine Dig A Hole To Put The Devil In (142-B-1),Lead Belly,1935,0,26,152947,0.876,0.668,0.916,0.005840,0.169,-8.105,1,A#,0.3930,124.605,0.671,"['acoustic blues', 'appalachian folk', 'blues'..."


# Pearson Correlation

In [50]:
from scipy.stats import pearsonr

In [51]:
count_matrix_array = count_matrix.toarray()

In [52]:
def recommend_pearson(id):
    idx = indices[indices == id].index[0]
    correlation = []
    for i in range(len(count_matrix_array)):
        correlation.append(pearsonr(count_matrix_array[idx], count_matrix_array[i])[0])
    correlation = list(enumerate(correlation))
    sorted_corr = sorted(correlation, reverse=True, key=lambda x: x[1])[1:6]
    songs_index = [i[0] for i in sorted_corr]
    return songs_df.iloc[songs_index]

In [55]:
recommend_pearson('Barking')

C:\Users\vtkurt13\anaconda3\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,name,artists,year,explicit,popularity,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,mode,key,speechiness,tempo,valence,genres
id,,,,,,,,,,,,,,,,,,
4GCADzGgIy4ywHIWCKkWNM,Yo Tuve un Cariño - Remasterizado,Francisco Canaro,1924,0,0,189547,0.995,0.456,0.1160,0.954,0.1460,-17.128,1,G#,0.1000,132.705,0.443,"['tango', 'vintage tango']"
1cuaa5iKOB69rkBD4sI6U6,Pussy Willow,Tommy Dorsey,1925,0,3,195320,0.923,0.590,0.2490,0.337,0.1800,-11.499,0,A#,0.0313,139.101,0.621,"['adult standards', 'big band', 'easy listenin..."
2lcTscCfqOZFl1cCiukP41,Amazon,Lionel Belasco,1930,0,3,177600,0.993,0.508,0.4000,0.649,0.3130,-14.734,1,A,0.0495,84.259,0.966,['calypso']
08oZMHCvq7oobnjWZXhzGu,The Firebird: VI Danse Infernale of King Kastc...,"Igor Stravinsky', 'Walter Straram Concerts Orc...",1930,0,0,232813,0.967,0.360,0.3180,0.791,0.1850,-15.734,1,E,0.0577,82.182,0.381,NaN
1UMdHJY4CSJkhXvbnyRqwu,Songs That Are Old Live Forever,"Henry Hall', 'The BBC Dance Orchestra",1932,0,4,164747,0.988,0.720,0.0897,0.275,0.0989,-17.535,1,G,0.0449,106.846,0.528,NaN


In [ ]:
songs_df.head()

In [ ]:
def recommend(index, method):
    id = indices[index]
    
    similar = list(enumerate(method[id]))
    similar = sorted(similar, key=lambda x: x[1], reverse=True)
    
    song_index = [i[0] for i in similar]
    
    return songs_10['id'].iloc[songs_index]